In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
import seaborn as sns

In [13]:
V52_data = pd.read_csv('V52_data.csv', parse_dates=['Date'])
V52_data = V52_data.set_index('Date')

# SUM all nan in the first column
#print('Total number of missing values in the first column: ', V52_data.iloc[:,0].isnull().sum())
# describe the data
print(V52_data.describe())

            Wsp_44m      Wdir_41m        ActPow         RePow    ActPow_std  \
count  35486.000000  35486.000000  35486.000000  35486.000000  35486.000000   
mean       6.376317    194.864858    199.143191      0.087797     51.864936   
std        2.589576     85.705501    219.936125      0.374143     48.046229   
min        0.539690      0.001286      0.000000     -4.625150      0.000000   
25%        4.595077    129.248500     47.158825     -0.001487     18.306925   
50%        5.840555    201.874000    112.923500     -0.000277     35.364950   
75%        7.668212    265.377500    266.188000      0.001519     73.348475   
max       19.720000    359.997000    867.900000      7.717470    408.458000   

        Wsp_44m_std  Wdir_41m_std     stability            TI          Hour  
count  3.548600e+04  35486.000000  35486.000000  3.548600e+04  52241.000000  
mean   9.513661e-01      8.380105      0.534718  1.514574e-01     11.499014  
std    5.067565e-01      4.180382      0.498800  5.909